In [1]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 43.1 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 895 kB 44.7 MB/s 
     |████████████████████████████████| 596 kB 42.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import torch
import torch.nn.functional as F 
from collections import namedtuple
import pandas as pd
from transformers import BertModel



In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
HIDDEN_DIM = 512
# 73 is temporary until we update templates
OUTPUT_DIM = 73
bert_model = BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
import torch.nn as nn

class BERTModel(nn.Module):
    def __init__(self,
                 bert_model,
                 hidden_dim,
                 output_dim,
                ):
        
        super().__init__()
        
        self.bert = bert_model
        
        embedding_dim = bert_model.config.to_dict()['hidden_size']
        
        self.out = nn.Linear(embedding_dim, output_dim)
        
        
    def forward(self, sequence, attn_mask, token_type):
        
        embedded = self.bert(input_ids = sequence, attention_mask = attn_mask, token_type_ids= token_type)[1]        
        
        output = self.out(embedded)
        
        return output

In [12]:
model = BERTModel(bert_model,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                        ).to(device)

model.load_state_dict(torch.load('temp_selection-4.pt'))

RuntimeError: ignored

In [ ]:
template_pool = pd.read_csv('./TemplatePool.csv')
Prediction = namedtuple('Prediction', 'label probability')


def select_template(context, model, device):
    
    model.eval()

    top_pred = Prediction(0, 0)
    
    context = '[CLS] ' + context + ' [SEP]'
    context_t = tokenize_bert(context)
    context_type = get_sent1_token_type(context_t)

    # Iter over all templates
    for _, item in template_pool.iterrows():
      template = item['Template']
      template = template  + ' [SEP]'
      template_t = tokenize_bert(template)
      template_type = get_sent2_token_type(template_t)

      indexes = context_t + template_t
      indexes = tokenizer.convert_tokens_to_ids(indexes)
      indexes_type = context_type + template_type
    
      attn_mask = get_sent2_token_type(indexes)

      indexes = torch.LongTensor(indexes).unsqueeze(0).to(device)
      indexes_type = torch.LongTensor(indexes_type).unsqueeze(0).to(device)
      attn_mask = torch.LongTensor(attn_mask).unsqueeze(0).to(device)
      
      # Predict
      predictions = F.softmax(model(indexes, attn_mask, indexes_type))

      curr_template_prob = predictions.max().item()
      # If highest prob, select
      if curr_template_prob > top_pred.probability:
        top_pred = Prediction(
            label=predictions.argmax(dim=-1).item(), 
            probability = curr_template_prob
        )


    return template_pool[template_pool['Class'] == top_pred.label]['Template'].values[0]

In [ ]:
ctx = "of an immense exuberance, organic in its form, kinetic, and drenched with the love of life... I rejoice over Mr. Wolfe.\" Both in his 1930 Nobel Prize for Literature acceptance speech and original press conference announcement, Sinclair Lewis, the first American to win the Nobel Prize for literature, said of Wolfe, \"He may have a chance to be the greatest American writer... In fact I don't see why he should not be one of the greatest world writers.\" Upon publication of his second novel, \"Of Time and the River\", most reviewers and the public remained supportive, though some critics found"

x = select_template(ctx, model, device)

In [ ]:
x